In [20]:
import pandas as pd
import numpy as np

import scipy
import statsmodels.stats.multitest as smm
from statsmodels.stats.weightstats import *

In [2]:
data = pd.read_csv("gene_high_throughput_sequencing.csv")
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
data.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,2.463922,7.100958,19.056151,15.618688,23.537120,11.303466,8.921396,63.270147,53.906324,70.678573,...,5.563444,2.192029,1.967424,2.181360,1.729308,1.980733,16.834075,1.825827,2.287380,1.756827
std,1.413687,4.164703,9.287753,4.664146,4.475294,4.055122,3.270713,13.792140,45.635498,21.326237,...,2.389789,2.454218,2.047129,2.542652,1.507149,2.230157,7.126802,1.902716,3.185571,1.623151
min,0.833898,1.231043,4.941211,6.395270,8.520713,2.066576,1.364917,42.544976,24.616767,42.631422,...,1.144240,0.853957,0.853957,0.853957,0.833898,0.833898,4.675683,0.833898,0.853957,0.833898
25%,1.284642,3.892403,13.423946,12.574596,21.151624,8.712898,6.213396,53.093627,34.009687,54.202100,...,3.805875,1.042783,1.011350,1.011350,1.003337,1.003337,11.186633,1.003337,1.011350,1.003337
50%,2.104677,6.315551,16.734855,14.586150,23.477006,11.172590,8.703397,63.230911,39.554954,65.510651,...,5.617824,1.339507,1.273861,1.298543,1.252527,1.252527,16.514389,1.252527,1.273861,1.252527
75%,3.345067,8.916399,21.884325,17.695678,26.471909,13.568625,11.139240,69.880705,50.295756,79.427207,...,7.100942,1.815754,1.647255,1.677971,1.625364,1.611299,21.849385,1.607345,1.647255,1.611299
max,7.364879,20.006038,60.584449,29.659104,34.110743,27.441093,17.121366,132.144503,327.590426,128.606260,...,11.227770,11.913855,10.883310,11.692697,10.392539,10.640754,49.295538,9.919132,17.278985,9.333904


In [11]:
print "Total number of records: ", data.shape[0]
data.Diagnosis.value_counts()

Total number of records:  72


early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [17]:
genes = np.array(data.columns)[2:]
print genes

['LOC643837' 'LOC100130417' 'SAMD11' ..., 'CYorf15B' 'KDM5D' 'EIF1AY']


In [18]:
normal = data[data.Diagnosis=='normal']
early_neo = data[data.Diagnosis=='early neoplasia']
cancer = data[data.Diagnosis=='cancer']

In [19]:
print len(normal)
print len(early_neo)
print len(cancer)

24
25
23


In [32]:
ttest_pvals1=[]
ttest_pvals2=[]
for gene in genes:
    ttest_pvals1.append(scipy.stats.ttest_ind(normal[gene], early_neo[gene], equal_var = False).pvalue)
    ttest_pvals2.append(scipy.stats.ttest_ind(early_neo[gene], cancer[gene], equal_var = False).pvalue)

In [33]:
alpha = 0.05
ttest_pvals1 = np.array(ttest_pvals1)
ttest_pvals2 = np.array(ttest_pvals2)

a11 = np.sum(ttest_pvals1<alpha)
a12 = np.sum(ttest_pvals2<alpha)

print a11, a12        

1575 3490


In [38]:
with open("w4-genes-1-1.txt", "w") as fout:
    fout.write(str(a11))
                   
with open("w4-genes-1-2.txt", "w") as fout:
    fout.write(str(a12))

In [41]:
rej1, pval_corr1 = smm.multipletests(ttest_pvals1, alpha=alpha/2, method='holm')[:2]
rej2, pval_corr2 = smm.multipletests(ttest_pvals2, alpha=alpha/2, method='holm')[:2]

In [42]:
def fold_change(C, T):
    res = 0
    if T>C:
        res = T/C
    elif C>T:
        res = C/T
    return res

In [44]:
c1=0
c2=0
for i in range(len(genes)):
    gene = genes[i]
    fc1 = fold_change(np.mean(normal[gene]),np.mean(early_neo[gene]))
    if np.abs(fc1)>1.5 and rej1[i]:
        c1+=1
    
    fc2 = fold_change(np.mean(early_neo[gene]),np.mean(cancer[gene]))
    if np.abs(fc2)>1.5 and rej2[i]:
        c2+=1
        
print c1, c2

2 77


In [45]:
with open("w4-genes-2-1.txt", "w") as fout:
    fout.write(str(c1))
                   
with open("w4-genes-2-2.txt", "w") as fout:
    fout.write(str(c2))

In [51]:
rej1, pval_corr1 = smm.multipletests(ttest_pvals1, alpha=alpha/2, method='fdr_bh')[:2]
rej2, pval_corr2 = smm.multipletests(ttest_pvals2, alpha=alpha/2, method='fdr_bh')[:2]

In [52]:
c1=0
c2=0
for i in range(len(genes)):
    gene = genes[i]
    fc1 = fold_change(np.mean(normal[gene]),np.mean(early_neo[gene]))
    if np.abs(fc1)>1.5 and rej1[i]:
        c1+=1
    
    fc2 = fold_change(np.mean(early_neo[gene]),np.mean(cancer[gene]))
    if np.abs(fc2)>1.5 and rej2[i]:
        c2+=1
        
print c1, c2

4 524


In [53]:
with open("w4-genes-3-1.txt", "w") as fout:
    fout.write(str(c1))
                   
with open("w4-genes-3-2.txt", "w") as fout:
    fout.write(str(c2))